In [1]:
%load_ext autoreload
%autoreload 1
import sys
sys.path.append(r'D:\\CodeRepository\\Python\\IDMSImplementation')
%aimport idms_impl.trajectory_generator
%aimport idms_impl.stop_area_mining
%aimport idms_impl.semantic_tag_conversion

In [2]:
orig_filename = r'C:\Users\Scott\Documents\ECNU\data\20201116\9_user.csv'

usecols = ['SUBSTR(MSISDN,1,7)', 'STAT_DATE', 'STIME', 'END_TIME', 'ZH_LABEL', 'LATITUDE', 'NUMBERITUDE', 'DURATION', 'DATA_TOTAL']
name_mapper = {
    "SUBSTR(MSISDN,1,7)": "USER_ID",
    "NUMBERITUDE": "LONGITUDE",
    "DATA_TOTAL": "TOTAL_DATA"
}
%run -m idms_impl.trajectory_generator

import time

start_time = time.time()

tr_gen = TrajectoryGenerator()

tr_gen.stop_area_mining(filename=orig_filename,
                        usecols=usecols,
                        name_mapper=name_mapper,
                        nan_dur_theta=4 * 3600,
                        dist_theta=5 / 111,
                        point_dur_theta=20,
                        eps=0.0036,
                        min_dur=900)
# tr_gen.sam.load_data(filename=orig_filename)

# tr_gen.sam.format_raw_data(usecols=usecols, name_mapper={
#     "SUBSTR(MSISDN,1,7)": "USER_ID",
#     "NUMBERITUDE": "LONGITUDE",
#     "DATA_TOTAL": "TOTAL_DATA"
# })

# tr_gen.sam.check_invalid_tr(nan_dur_theta=4 * 3600, dist_theta=5 / 111)

# tr_gen.sam.gen_valid_area(point_dur_theta=20, eps=0.0036, min_dur=900)

# tr_gen.sam.df.to_csv('../data/no_invalid_area.csv', encoding='gbk', index=False)
# tr_gen.sam.merge_adjacent_points()

# tr_gen.sam.df.to_csv('../data/sam.csv', encoding='gbk', index=False)

end_time = time.time()
print(f'Finish it in {end_time - start_time}s')

Finish it in 8.303912878036499s


In [193]:
# poi_gen.knn_model.kneighbors([[31.96745185244962, 118.71106705349086]], return_distance=False)
from collections import Counter
def handle_cluster(cluster):

    def get_ctg(base_name):
        for ctg in ctgs:
            if base_name == ctg[0]:
                return ctg
    
    def find_poi():
        if cluster_type == 'home':
            for name, cnt in poi_counter:
                if name in home_list:
                    return get_ctg(name)
        elif cluster_type == 'work':
            for name, cnt in poi_counter:
                if name in work_list:
                    return get_ctg(name)
        return get_ctg(poi_counter[0][0])

    coords = cluster[:2].values.reshape(1, 2)
    cluster_type = cluster[-1]
    index = poi_gen.knn_model.kneighbors(coords, return_distance=False).flatten()
    ctgs = poi_gen.poi_df.iloc[index, :7].values
    # print(ctg)
    c = Counter(ctgs[:, 0])
    cluster_type = cluster[-1]
    poi_counter = c.most_common()
    home_list = ['商务住宅', '住宿服务', '生活服务', '地名地址信息']
    work_list = ['公司企业']
    poi = find_poi()
    # print("+" * 10)
    # print(poi_counter)
    # print(ctgs)
    # print(poi)
    # cluster[['big_ctg', 'medium_ctg', 'small_ctg', 'name']] = pd.Series( poi)
    cluster['big_ctg'] = poi[0]
    cluster['medium_ctg'] = poi[1]
    cluster['small_ctg'] = poi[2]
    cluster['name'] = poi[3]
    cluster['province'] = poi[4]
    cluster['city'] = poi[5]
    cluster['region'] = poi[6]
    return cluster
new_cluster_poi = cluster_poi.apply(handle_cluster, axis=1)

# cluster_poi.index.values

In [196]:
new_df = df.copy()
new_df.set_index(['USER_ID', 'CLUSTER_ID'], inplace=True)
new_df[['type', 'big_ctg', 'medium_ctg', 'small_ctg', 'name', 'province', 'city', 'region']] = new_cluster_poi.iloc[:, 2:]


In [198]:
# new_df.reset_index(drop=False, inplace=True)
new_df
new_df.to_csv('../data/9_user_with_flag.csv', encoding='gbk', index=False)

In [3]:
tr_gen.semantic_tr_convertion()
tr_gen.stc.main_area_mining(0.9)

In [68]:
user_grp = df.groupby(['USER_ID'], sort=False)

def gen_cluster_attr(user):
    cluster_grp = user.groupby(['CLUSTER_ID'], sort=False)
    attr = cluster_grp.apply(lambda x: pd.Series([x['STAT_DATE'].nunique(), x['DURATION'].sum(), x['TOTAL_DATA'].sum() / x['STAT_DATE'].nunique()], index=['sn', 'dur_sum', 'd']))
    attr.sort_values('dur_sum', ascending=False, inplace=True)
    return attr

cluster_attr = user_grp.apply(gen_cluster_attr)
cluster_attr['type'] = np.array(['other'] * cluster_attr.shape[0])

def home_prob(cid, n):
    sn = cluster_attr.loc[cid, 'sn']
    sn_prob = sn / n 
    d = cluster_attr.loc[cid, 'd']
    sum_d = cluster_attr.loc[cid[0], 'd'].sum()
    d_prob = 0. if sum_d == 0. else d / sum_d
    return sn_prob + (1 - d_prob)

def gen_main_area(user):
    
    n = user['STAT_DATE'].nunique()
    uid = user.iloc[0, 0]
    dur_sum_in_user = cluster_attr.loc[uid, 'dur_sum'].sum()
    
    cur_cluster_attr = cluster_attr.loc[uid]

    sr_list = []
    tmp_dur_sum = 0
    for cluster_id, row in cur_cluster_attr.T.iteritems():
        tmp_dur_sum += row['dur_sum']
        if tmp_dur_sum / dur_sum_in_user <= theta:
            sr_list.append((uid, cluster_id))
    n_sr = len(sr_list)
    # print(sr_list)

    if n_sr == 1:
        cluster_attr.loc[sr_list[0], 'type'] = 'home'
    elif n_sr == 2:
        if home_prob(sr_list[0], n) > home_prob(sr_list[1], n):
            cluster_attr.loc[sr_list[0], 'type'] = 'home'
            cluster_attr.loc[sr_list[1], 'type'] = 'work'
        else:
            cluster_attr.loc[sr_list[0], 'type'] = 'work'
            cluster_attr.loc[sr_list[1], 'type'] = 'home'
    elif n_sr == 3:
        d0 = cluster_attr.loc[sr_list[0], 'd']
        d1 = cluster_attr.loc[sr_list[1], 'd']
        d2 = cluster_attr.loc[sr_list[2], 'd']
        if d0 > (d1 + d2) / 2:
            cluster_attr.loc[sr_list[0], 'type'] = 'work'
            cluster_attr.loc[sr_list[1], 'type'] = 'home'
            cluster_attr.loc[sr_list[2], 'type'] = 'home'
        else:
            cluster_attr.loc[sr_list[0], 'type'] = 'home'
            cluster_attr.loc[sr_list[1], 'type'] = 'work'
            cluster_attr.loc[sr_list[2], 'type'] = 'work'
        
theta = 0.9

# for uid, user in user_grp:
#     gen_main_area(user)
user_grp.apply(gen_main_area)


""


In [81]:
cluster_attr

sn   dur_sum              d   type
USER_ID CLUSTER_ID                                      
1377032 0           21.0  858715.0   57049.663830   home
        2           15.0  619153.0  238137.694661   work
        22           3.0   96684.0  228284.142578  other
        6            1.0   42723.0  433745.323242  other
        10           1.0   34091.0  137122.463867  other
...                  ...       ...            ...    ...
1870519 5            1.0    2076.0    1552.216797  other
        22           1.0    2042.0  250869.634766  other
        17           1.0    1639.0       0.000000  other
        16           1.0    1359.0       0.000000  other
        28           1.0     958.0     872.636719  other

[160 rows x 4 columns]

In [ ]:

# cluster_grp = df.groupby(['USER_ID', 'CLUSTER_ID'], sort=False)
# def func(cluster):
#     series = pd.Series([cluster['STAT_DATE'].nunique(), cluster['DURATION'].sum(), cluster['TOTAL_DATA'].sum() / cluster['STAT_DATE'].nunique()], index=['sn', 'dur_sum', 'd'])
#     return series
# cluster_attr = cluster_grp.apply(func)


user_grp = df.groupby(['USER_ID'], sort=False)

def func(user):

    c_grp = user.groupby(['CLUSTER_ID'], sort=False)
    
    attr = c_grp.apply(lambda x: pd.Series([x['STAT_DATE'].nunique(), x['DURATION'].sum(), x['TOTAL_DATA'].sum() / x['STAT_DATE'].nunique()], index=['sn', 'dur_sum', 'd']))

    attr.sort_values('dur_sum', ascending=False, inplace=True)
    return attr

cluster_attr = user_grp.apply(func)
cluster_attr.head(50)
# cluster_attr.index

# print(cluster_attr.index.names)
# cluster_attr.loc['USER_ID']
# cluster_attr[['sn', 'dur_sum']] = cluster_attr[['sn', 'dur_sum']].astype(int)

# cluster_attr.sort_values('dur_sum', ascending=False, inplace=True)
# cluster_attr.head(50)
# cluster_attr.loc[(1377032, 0), 'sn']
# for cid, cluster in cluster_grp:
#     # print(cluster['STAT_DATE'])
#     print(cluster['STAT_DATE'].nunique())
#     # print(cluster['DURATION'])
#     print(cluster['DURATION'].sum())
#     print(cluster['TOTAL_DATA'].sum())
#     print(cluster['TOTAL_DATA'].sum() / cluster['STAT_DATE'].nunique())
#     break

In [ ]:
from typing import Tuple
def home_prob(cid, n):
    sn = cluster_attr.loc[cid, 'sn']
    sn_prob = sn / n 
    d = cluster_attr.loc[cid, 'd']
    sum_d = cluster_attr.loc[cid[0], 'd'].sum()
    d_prob = 0. if sum_d == 0. else d / sum_d
    print(d_prob)
    print(sn_prob)
    return sn_prob + (1 - d_prob)
    # sn_prob = cluster_attr.loc[cid, 'sn']
home_prob((1377032, 0), 21)
# print(cluster_attr.loc[1377032, 'd'].sum())


In [ ]:
user_grp = df.groupby(['USER_ID'], sort=False)

for uid, user in user_grp:
    # c_grp = user.groupby(['CLUSTER_ID'], sort=False)
    cluster = cluster_attr.loc[uid]
    sss = cluster['dur_sum'].sum()
    print(sss)
    dur_sums = cluster['dur_sum'].values
    # print(dur_sums)
    # print(cluster.iloc[[0,1, 2]].index)
    dd = 0
    for i, dur_sum in enumerate( dur_sums):
        print(i, dur_sum)
    break

In [127]:
%run ../idms_impl/poi.py
poi_gen = POIGenerator()
poi_gen.load_poi(filename=r'C:\\Users\\Scott\\Documents\\ECNU\\data\\poi\\nj_poi.csv')
poi_gen.train_data(4)
index = poi_gen.knn_model.kneighbors([[31.967434, 118.711093]], return_distance=False).flatten()


[[ 32.57567089 118.78719721]
 [ 32.56770851 118.78874347]
 [ 32.56205344 118.87286009]
 ...
 [ 31.24232641 118.84930304]
 [ 31.24232954 118.84938618]
 [ 31.24267898 119.0790029 ]]


In [128]:
from collections import Counter
print(index)
ctg = poi_gen.poi_df.iloc[index, :3].values
c = Counter(ctg[:, 0])
c.most_common()

[100990  31120  69968  22312]


[('商务住宅', 1), ('体育休闲服务', 1), ('公司企业', 1), ('住宿服务', 1)]

In [76]:
%run ../demo/algorithm.py
df = df.iloc[:2]
df
df[['E', 'F']] = [3, 3]
df

,A,B,C,D,E,F
0,foo,one,-0.575247,1.346061,3,3
1,bar,one,0.254161,1.511763,3,3


In [80]:
import pandas as pd
import numpy as np 
s = pd.Series({'A': 1, 'B':2})
# s[['E', 'F']] = [3, 4]
s.append(pd.Series({'E': 3, 'F':4}))

A    1
B    2
E    3
F    4
dtype: int64

In [37]:
from collections import Counter

c = Counter([1, 1, 2, 3, 4])
a = np.array([1, 2, 3, 3])
print(c.keys())
np.where(a == c.keys())

dict_keys([1, 2, 3, 4])


(array([], dtype=int64),)

In [46]:
ses = pd.Series([1, 2], index=['A', 'B'])
poi = np.array([3, 4, 5])
poi_ses = pd.Series(poi, index=['C', 'D', 'E'])
ses.append(poi_ses)

A    1
B    2
C    3
D    4
E    5
dtype: int64